# Heat Pump

- Theorie 2
   - Verbesserung des COP -> Andere Topologie
- Übung 2
   - Bauen der WP
   - Warum verbesser es den Wirkungsgrad?
      - Verlgeich Aufwand und Nutzenströme bei gleichem Nennwärmestrom
      - Vergleich der Komponentenverluste
      - ...

## Introduction

```{figure} /figures/heat_pump.svg
---
name: heat-pump-flowsheet
---
Flow sheet of the simplest implementation of a heat pump.
```

As the final simple thermodynamic cycle, a heat pump will be subject of the exergy analysis. Heat pumps are a promising technology in the process of climate change mitigation, because they can replace conventional fossil fuel based heating plants. As heat pumps are scalable, they can be used in households and building, as well as whole neighbourhoods and in district heating systems. A heat pump employs the well known refrigeration cycle (see {numref}`heat-pump-flowsheet`), but instead uses the heat that is transfered out of the system. For the cycle to work, heat has to be transferred into the system at a lower temperature level and through the input of work can be transferred out on a higher temperature level. The resulting energy balance of a heat pump is displayed in Eq. {eq}`heat-pump-energy-balance`.

```{math}
    :label: heat-pump-energy-balance
    0 = Q_{in} + Q_{out} + W_{in}
```

As traditionally is done with heat engines, the benefit of a cycle can be put in to relation with the necessary input. Formalizing this expression yields the so called coefficient of performance of a heat pump, as is shown in Eq. {eq}`heat-pump-coefficient-of-performance`. From the energy balance {eq}`heat-pump-energy-balance` we can infer, that the outgoing heat is the sum of the work and heat put in. This means, that the $COP$ must be greater than {math}`1`, in contrast to the thermal efficiency of heat engines.

```{math}
    :label: heat-pump-coefficient-of-performance
    COP = \frac{Q_{out}}{W_{in}}
```

Coming from thermal efficiencies of heat engines, this can seem to be in conflict the laws of thermodynamics. We get more heat from the cycle than work needs to be put in. There only has to be an ambient heat source supply heat at a lower temperature. To reveal the thermodynamic losses occuring in this process and to show that the cycle is no perpetuum mobile, we can analyze the exergy flows.

At first, we have to define the fuel and the product of the heat pump cycle. As described above, the benefit of the cycle is the heat transfered out of the system, which makes it the product. The necessary fuels logically are the heat and work inputs. Added to the compressor work are necessary supplemental work inputs like recirculation pumps. With these defined, we can start to build the heat pump model.

## Exercise 1

1. Build a simple heat pump model using ammonia with TESPy according to the flowsheet in {numref}`heat-pump-flowsheet`, but add a water fed heat source (the hot side of the evaporator) and a consumer cycle as the heat sink (the cold side of the condenser). Parametrize the model according to the description as shown in {numref}`heat-pump-param`.
2. Execute the exergy analysis for the heat pump model. Visualize how each component destroyes part of the fuel exergy through a waterfall diagram. Analyze the influence of the compressors isentropic efficiency, as well as that of the heat exchangers terminal temperature differences.
3. Analyze the impact of the ambient temperature level on the components and total efficiency.

```{list-table} Parametrization of the heat pump
:header-rows: 1
:name: heat-pump-param

* - Description
  - Parameter
  - Value
  - Unit
* - Compressor
  - Isentropic efficiency
  - 75
  - \%
* - 
  - Motor efficiency
  - 96
  - \%
* - Pumps
  - Isentropic efficiency
  - 80
  - \%
* - 
  - Motor efficiency
  - 96
  - \%
* - All Heat Exchangers
  - Pressure losses
  - 2
  - \%
* - 
  - Terminal temperature difference
  - 2
  - K
* - Heat Sink
  - Nominal heat demand
  - -500
  - kW
* - 
  - Feed flow temperature
  - 70
  - °C
* - 
  - Back flow temperature
  - 50
  - °C
* - 
  - System pressure
  - 10
  - bar
* - Heat Source
  - Incoming temperature
  - 10
  - °C
* - 
  - Temperature reduction
  - 5
  - K
* - 
  - System pressure
  - 1.013
  - bar
```

```{note}

If you are heaving trouble with the simulation not converging, try setting best guesses for pressures instead of terminal temperature differences and only after a succesful simulation reset the intended values (see also the TESPy tutorial on 
<a href="https://tespy.readthedocs.io/en/dev/tutorials/starting_values.html#tespy-tutorial-starting-values-label">How to Generate Stable Starting Values</a>).
```

```{note}

If you are encountering problems with specific components during the exergy analysis, try having a look at the individual components <a href="https://tespy.readthedocs.io/en/latest/api.html">API documentation</a> and their respective implementation of the `exergy_balance` method.
```

### Proposed solution 1.1

**First step:** Build the heat pump model topology

In [1]:
from tespy.networks import Network
from tespy.connections import Connection, Bus
from tespy.components import (
    CycleCloser, Source, Sink, Pump, HeatExchanger, Condenser,
    HeatExchangerSimple, Compressor, Valve
    )

# Network
refrigerant = 'Ammonia'
nw = Network(
    fluids=[refrigerant, 'water'],
    T_unit='C', p_unit='bar', h_unit='kJ / kg', m_unit='kg / s'
)

# Components
cycle_closer = CycleCloser('Refrigerant Cycle Closer')

heatsource_ff = Source('Heat Source Feed Flow')
evaporator = HeatExchanger('Evaporator')
heatsource_pump = Pump('Heat Source Recirculation Pump')
heatsource_bf = Sink('Heat Source Back Flow')

compressor = Compressor('Compressor')

cons_cc = CycleCloser('Consumer Cycle Closer')
cons_pump = Pump('Heat Sink Recirculation Pump')
condenser = Condenser('Condenser')
cons_heatsink = HeatExchangerSimple('Heat Consumer')

valve = Valve('Expansion Valve')

# Connections
cc2evap = Connection(cycle_closer, 'out1', evaporator, 'in2', label='00')
hs_ff2evap = Connection(heatsource_ff, 'out1', evaporator, 'in1', label='10')
evap2hs_pump = Connection(evaporator, 'out1', heatsource_pump, 'in1', label='11')
hs_pump2hs_bf = Connection(heatsource_pump, 'out1', heatsource_bf, 'in1', label='12')

nw.add_conns(cc2evap, hs_ff2evap, evap2hs_pump, hs_pump2hs_bf)

evap2comp = Connection(evaporator, 'out2', compressor, 'in1', label='01')
comp2cond = Connection(compressor, 'out1', condenser, 'in1', label='02')

nw.add_conns(evap2comp, comp2cond)

cons_cc2cons_pump = Connection(cons_cc, 'out1', cons_pump, 'in1', label='20')
cons_pump2cond = Connection(cons_pump, 'out1', condenser, 'in2', label='21')
cond2cons_hs = Connection(condenser, 'out2', cons_heatsink, 'in1', label='22')
cons_hs2cons_cc = Connection(cons_heatsink, 'out1', cons_cc, 'in1', label='23')

nw.add_conns(cons_cc2cons_pump, cons_pump2cond, cond2cons_hs, cons_hs2cons_cc)

cond2valve = Connection(condenser, 'out1', valve, 'in1', label='03')
valve2cc = Connection(valve, 'out1', cycle_closer, 'in1', label='04')

nw.add_conns(cond2valve, valve2cc)



**Second step:** Set the heat pumps process parameters and starting values and define the energy busses

In [2]:
from CoolProp.CoolProp import PropsSI as PSI

# Component parameters
compressor.set_attr(eta_s=0.75)
cons_pump.set_attr(eta_s=0.8)
heatsource_pump.set_attr(eta_s=0.8)

evaporator.set_attr(pr1=0.98, pr2=0.98)
condenser.set_attr(pr1=0.98, pr2=0.98)
cons_heatsink.set_attr(pr=0.98, Q=-500e3, dissipative=False)

# Connection parameters
cond2cons_hs.set_attr(T=70, p=10, fluid={refrigerant: 0, 'water': 1})
cons_hs2cons_cc.set_attr(T=50)

hs_ff2evap.set_attr(T=10, p=1.013, fluid={refrigerant: 0, 'water': 1})
evap2hs_pump.set_attr(T=5)
hs_pump2hs_bf.set_attr(p=1.013)

evap2comp.set_attr(x=1, fluid={refrigerant: 1, 'water': 0})

# Starting values
p_evap = PSI('P', 'Q', 1, 'T', 5 - 2 + 273.15, refrigerant)
evap2comp.set_attr(p=p_evap*1e-5)
p_cond = PSI('P', 'Q', 0, 'T', 70 + 2 + 273.15, refrigerant)
cond2valve.set_attr(p=p_cond*1e-5)

# Busses
heat_in = Bus('Heat Input')
heat_in.add_comps(
    {'comp': heatsource_ff, 'base': 'bus'},
    {'comp': heatsource_bf, 'base': 'component'}
    )

heat_out = Bus('Heat Output')
heat_out.add_comps(
    {'comp': cons_heatsink, 'base': 'component'})

power_in = Bus('Power Input')
power_in.add_comps(
    {'comp': compressor, 'char': 0.96, 'base': 'bus'},
    {'comp': heatsource_pump, 'char': 0.96, 'base': 'bus'},
    {'comp': cons_pump, 'char': 0.96, 'base': 'bus'}
    )

nw.add_busses(heat_in, heat_out, power_in)


**Third step:** Solve model with starting and final values

In [3]:
from tespy.tools import ExergyAnalysis

# Initial solve with starting values
nw.set_attr(iterinfo=False)
nw.solve('design')

# Final solve 
evap2comp.set_attr(p=None)
cond2valve.set_attr(p=None)
evaporator.set_attr(ttd_l=2)
condenser.set_attr(ttd_u=2)

nw.solve('design')
print(f'COP = {abs(heat_out.P.val)/power_in.P.val:.3f}')


iter	| residual | massflow | pressure | enthalpy | fluid
--------+----------+----------+----------+----------+---------
1	| 8.50e+06 | 2.68e+02 | 6.77e+06 | 1.95e+06 | 0.00e+00
2	| 8.58e+06 | 2.44e+02 | 2.50e+06 | 2.40e+04 | 0.00e+00
3	| 1.09e+06 | 2.50e+00 | 7.95e+05 | 2.70e-01 | 0.00e+00
4	| 1.90e+05 | 9.88e-06 | 1.36e+05 | 2.08e-02 | 0.00e+00
5	| 3.69e-08 | 2.24e-10 | 0.00e+00 | 2.62e-07 | 0.00e+00
--------+----------+----------+----------+----------+---------
Total iterations: 5, Calculation time: 0.1 s, Iterations per second: 35.74
iter	| residual | massflow | pressure | enthalpy | fluid
--------+----------+----------+----------+----------+---------
1	| 1.05e+00 | 4.84e-02 | 1.01e+05 | 8.46e+03 | 0.00e+00
2	| 2.94e+01 | 1.72e-04 | 7.61e+02 | 1.70e+01 | 0.00e+00
3	| 1.69e-03 | 3.35e-09 | 4.41e-02 | 9.71e-04 | 0.00e+00
4	| 8.39e-07 | 9.47e-11 | 1.75e-09 | 9.82e-07 | 0.00e+00
5	| 9.99e-07 | 9.89e-11 | 1.75e-09 | 1.32e-06 | 0.00e+00
--------+----------+----------+----------+----------

### Proposed solution 1.2

**First step:** Create and execute `ExergyAnalysis` instance and prepare result data

In [ ]:
from tespy.tools import ExergyAnalysis

ean = ExergyAnalysis(network=nw, E_F=[heat_in, power_in], E_P=[heat_out])
ean.analyse(pamb=1.013, Tamb=10)
ean.print_results(groups=False, connections=False, aggregation=False)

comps = ['Fuel Exergy']
E_F = ean.network_data.E_F
E_D = [0]
E_P = [E_F]
for comp in ean.component_data.index:
    # only plot components with exergy destruction > 1 W
    if ean.component_data.E_D[comp] > 1 :
        comps.append(comp)
        E_D.append(ean.component_data.E_D[comp])
        E_F = E_F - ean.component_data.E_D[comp]
        E_P.append(E_F)
comps.append('Product Exergy')
E_D.append(0)
E_P.append(E_F)

E_D = [E * 1e-3 for E in E_D]
E_P = [E * 1e-3 for E in E_P]


**Second step:** Plot the waterfall diagram of the heat pump process

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

colors_E_P = ['#74ADC0'] * len(comps)
colors_E_P[0] = '#00395B'
colors_E_P[-1] = '#B54036'

fig, ax = plt.subplots(figsize=(8, 6))

ax.barh(np.arange(len(comps)), E_P, align='center', color=colors_E_P)
ax.barh(np.arange(len(comps)), E_D, align='center', left=E_P, label='E_D', color='#EC6707')

ax.legend()
ax.annotate(
    f'$\epsilon_{{tot}} = ${E_P[-1]/E_P[0]:.3f}', (0.65, 0.05),
    xycoords='axes fraction',
    ha='left', va='center', color='k',
    bbox=dict(boxstyle='round,pad=0.3', fc='white')
    )
ax.set_xlabel('Exergy in kW')
ax.set_yticks(np.arange(len(comps)))
ax.set_yticklabels(comps)
ax.invert_yaxis()
ax.grid(axis='x')
ax.set_axisbelow(True)

**Third step:** Variate compressor isentropic efficiency and terminal temperature differences of evaporator and condenser

In [14]:
def get_ean_results(ean):
    comps = ['Fuel Exergy', 'Expansion Valve', 'Evaporator', 'Compressor', 'Condenser', 'Product Exergy']
    E_F = ean.network_data.E_F * 1e-3
    E_D = [0]
    E_P = [E_F]

    for comp in comps[1:-1]:
        E_D += [ean.component_data.E_D[comp] * 1e-3]
        E_F -= E_D[-1]
        E_P += [E_F]

    E_D += [0]
    E_P += [E_F]

    return E_P, E_D, comps

eta_s_range = np.linspace(0.65, 0.8, 4)
ttd_range = np.arange(2, 10, 2)

fig, axs = plt.subplots(
    len(ttd_range), len(eta_s_range),
    sharex=True, sharey=True, figsize=(14, 14)
    )
axs[0, 0].invert_yaxis()

for col, eta_s in enumerate(eta_s_range):
    compressor.set_attr(eta_s=eta_s)
    for row, ttd in enumerate(ttd_range):
        evaporator.set_attr(ttd_l=ttd)
        condenser.set_attr(ttd_u=ttd)

        nw.solve('design')
        ean = ExergyAnalysis(network=nw, E_F=[heat_in, power_in], E_P=[heat_out])
        ean.analyse(pamb=1.013, Tamb=10)

        E_P, E_D, comps = get_ean_results(ean)

        colors_E_P = ['#74ADC0'] * len(comps)
        colors_E_P[0] = '#00395B'
        colors_E_P[-1] = '#B54036'

        axs[row, col].barh(
            np.arange(len(comps)), E_P, align='center', color=colors_E_P
            )
        axs[row, col].barh(
            np.arange(len(comps)), E_D, align='center', left=E_P, label='E_D', color='#EC6707'
            )

        axs[row, col].set_yticks(np.arange(len(comps)))
        axs[row, col].set_yticklabels(comps)
        axs[row, col].grid(axis='x')
        axs[row, col].set_axisbelow(True)

        axs[row, col].annotate(
                    f'$\epsilon_{{tot}} = ${E_P[-1]/E_P[0]:.3f}', (0.65, 0.05),
                    xycoords='axes fraction',
                    ha='left', va='center', color='k',
                    bbox=dict(boxstyle='round,pad=0.3', fc='white')
                    )

        if row == len(ttd_range)-1:
            axs[row, col].set_xlabel(f'Isentropic Efficiency {eta_s:.2f}')
        if col == 0:
            axs[row, col].set_ylabel(f'Terminal temperature difference {ttd} K')

array([2, 4, 6, 8])